In [1]:
!pip3 install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.9 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import open3d as o3d
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import glob
from scipy.spatial import cKDTree
import warnings
import torch.nn.functional as F


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import numpy as np
import glob

def load_preprocessed_pairs(input_dir):
    pairs = []
    npz_files = sorted(glob.glob(os.path.join(input_dir, '*.npz')))  # pair_0.npz, pair_1.npz...
    for file in npz_files:
        data = np.load(file)
        static = data['static']
        dynamic = data['dynamic']
        pairs.append((static, dynamic))
    return pairs

pairs = load_preprocessed_pairs('/kaggle/input/kitti360-preprocessed-pairs')
print(f"Загружено {len(pairs)} pairs")

Загружено 205 pairs


In [4]:

split_idx = int(0.9 * len(pairs))
train_pairs = pairs[:split_idx]
val_pairs = pairs[split_idx:]
print(len(train_pairs), len(val_pairs))

184 21


In [ ]:
class KittiDynamicDataset(Dataset):
    def __init__(self, pairs, num_points=4096, voxel_size=0.05, dynamic_r=0.1, is_train=True):
        self.pairs = pairs
        self.num_points = num_points
        self.voxel_size = voxel_size
        self.dynamic_r = dynamic_r
        self.is_train = is_train
        self.dynamic_ratio = 0.7 if is_train else None

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        static, dynamic = self.pairs[idx]

        if len(static) > 0 and len(dynamic) > 0:
            tree = cKDTree(dynamic)
            dist, _ = tree.query(static, k=1)
            keep_mask = dist > self.dynamic_r
            static_clean = static[keep_mask]
        else:
            static_clean = static

        if len(dynamic) > 0:
            all_points = np.vstack([static_clean, dynamic])
            labels = np.hstack([
                np.zeros(len(static_clean), dtype=np.int64),
                np.ones(len(dynamic), dtype=np.int64)
            ])
        else:
            all_points = static_clean
            labels = np.zeros(len(static_clean), dtype=np.int64)

        if self.is_train and self.dynamic_ratio is not None and self.dynamic_ratio > 0:
            num_dynamic = int(self.num_points * self.dynamic_ratio)
            num_static = self.num_points - num_dynamic

            if len(dynamic) > 0:
                dynamic_choice = np.random.choice(len(dynamic), num_dynamic, replace=True)
                dynamic_pts = dynamic[dynamic_choice]
                dynamic_lbls = np.ones(num_dynamic, dtype=np.int64)
            else:
                dynamic_pts = np.empty((0, dynamic.shape[1])) 
                dynamic_lbls = np.array([])
                num_dynamic = 0 
                num_static = self.num_points 

            # Undersample static
            if len(static_clean) >= num_static:
                static_choice = np.random.choice(len(static_clean), num_static, replace=False)
            else:
                static_choice = np.random.choice(len(static_clean), num_static, replace=True)  # Уже есть, ок для мало static
            static_pts = static_clean[static_choice]
            static_lbls = np.zeros(num_static, dtype=np.int64)

            all_points = np.vstack([static_pts, dynamic_pts])
            labels = np.hstack([static_lbls, dynamic_lbls])

            perm = np.random.permutation(self.num_points)
            pts = all_points[perm]
            lbls = labels[perm]

        else: 
            if len(all_points) >= self.num_points:
                choice = np.random.choice(len(all_points), self.num_points, replace=False)
            else:
                choice = np.random.choice(len(all_points), self.num_points, replace=True)
            pts = all_points[choice]
            lbls = labels[choice]
            perm = np.random.permutation(self.num_points)
            pts = pts[perm]
            lbls = lbls[perm]
        # pts = all_points[choice]
        # lbls = labels[choice]
        # print(len(pts))
        xyz = pts[:, :3] 
        xyz_mean = np.mean(xyz, axis=0, keepdims=True) 
        xyz = xyz - xyz_mean        
        xyz_max = np.max(np.abs(xyz))
        xyz = xyz / (xyz_max + 1e-6) 

        pts[:, :3] = xyz

        rgb = pts[:, 3:6]
        if np.max(rgb) > 1.0: 
            rgb = rgb / 255.0

        rgb_mean = np.mean(rgb, axis=0, keepdims=True)
        rgb_std = np.std(rgb, axis=0, keepdims=True) + 1e-6
        rgb = (rgb - rgb_mean) / rgb_std
        pts[:, 3:6] = rgb
        
        # print(pts.shape)
        
        # if pts.shape[1] < 9:
        #     pad_width = 9 - pts.shape[1]
        #     pts = np.hstack([pts, np.ones((self.num_points, pad_width))])

        # aug
        if self.is_train and len(dynamic) > 0:
            jitter = np.random.normal(0, 0.02, size=(num_dynamic, 3))
            dynamic_pts[:, :3] += jitter  

        # print(pts)
        return torch.tensor(pts, dtype=torch.float32), torch.tensor(lbls, dtype=torch.long)


In [6]:
!git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git
%cd Pointnet_Pointnet2_pytorch

Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Total 842 (delta 0), reused 0 (delta 0), pack-reused 842 (from 1)
Receiving objects: 100% (842/842), 68.77 MiB | 38.21 MiB/s, done.
Resolving deltas: 100% (485/485), done.
/kaggle/working/Pointnet_Pointnet2_pytorch


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [8]:
import torch
import torch.nn as nn
from models.pointnet2_sem_seg import get_model, get_loss

num_classes = 2 
model = get_model(num_classes=num_classes).to(device) #[B, num_classes, N]

In [9]:
model_path = "/kaggle/input/lidarsegmmodel-17/best_model.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))

<All keys matched successfully>

In [10]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss.sum()

In [11]:
def iou_loss(pred, target, num_classes=2, smooth=1e-6):
    """
    Smooth IoU loss Pred: [B*N, C] после softmax, target: [B*N] long.
    """
    pred = F.softmax(pred, dim=1)  # [B*N, C]
    one_hot_target = F.one_hot(target, num_classes=num_classes).float()  # [B*N, C]
    
    intersection = (pred * one_hot_target).sum(dim=0)  # По классам [C]
    union = (pred + one_hot_target).sum(dim=0) - intersection  # [C]
    
    iou = (intersection + smooth) / (union + smooth)
    return 1 - iou.mean() 

In [12]:
from __future__ import print_function, division

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
try:
    from itertools import ifilterfalse
except ImportError:  # py3k
    from itertools import filterfalse as ifilterfalse

def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    p = len(gt_sorted)
    gts = gt_sorted.sum()
    intersection = gts - gt_sorted.float().cumsum(0)
    union = gts + (1 - gt_sorted).float().cumsum(0)
    jaccard = 1. - intersection / union
    if p > 1:  # cover 1-pixel case
        jaccard[1:p] = jaccard[1:p] - jaccard[0:-1]
    return jaccard

def lovasz_softmax_flat(probas, labels, classes='present'):
    """
    Multi-class Lovasz-Softmax loss
      probas: [P, C] Variable, class probabilities at each prediction (between 0 and 1)
      labels: [P] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
    """
    if probas.numel() == 0:
        # only void pixels, the gradients should be 0
        return probas * 0.
    C = probas.size(1)
    losses = []
    class_to_sum = list(range(C)) if classes in ['all', 'present'] else classes
    for c in class_to_sum:
        fg = (labels == c).float()  # foreground for class c
        if (classes == 'present' and fg.sum() == 0):
            continue
        if C == 1:
            if len(classes) > 1:
                raise ValueError("Sigmoid output possible only with 1 class")
            class_pred = probas[:, 0]
        else:
            class_pred = probas[:, c]
        errors = (Variable(fg) - class_pred).abs()
        errors_sorted, perm = torch.sort(errors, 0, descending=True)
        perm = perm.data
        fg_sorted = fg[perm]
        losses.append(torch.dot(errors_sorted, Variable(lovasz_grad(fg_sorted))))
    return mean(losses)

def flatten_probas(probas, labels, ignore=None):
    """
    Flattens predictions in the batch
    """
    if probas.dim() == 3:
        # assumes output of a sigmoid layer
        B, H, W = probas.size()
        probas = probas.view(B, 1, H, W)
    B, C, H, W = probas.size()
    probas = probas.permute(0, 2, 3, 1).contiguous().view(-1, C)  # B * H * W, C = P, C
    labels = labels.view(-1)
    if ignore is None:
        return probas, labels
    valid = (labels != ignore)
    vprobas = probas[valid.nonzero().squeeze()]
    vlabels = labels[valid]
    return vprobas, vlabels

def lovasz_softmax(probas, labels, classes='present', per_image=False, ignore=None):
    """
    Multi-class Lovasz-Softmax loss
      probas: [B, C, H, W] Variable, class probabilities at each prediction (between 0 and 1).
              Interpreted as binary (sigmoid) output with outputs of size [B, H, W].
      labels: [B, H, W] Tensor, ground truth labels (between 0 and C - 1)
      classes: 'all' for all, 'present' for classes present in labels, or a list of classes to average.
      per_image: compute the loss per image instead of per batch
      ignore: void class labels
    """
    if per_image:
        loss = mean(lovasz_softmax_flat(*flatten_probas(prob.unsqueeze(0), lab.unsqueeze(0), ignore), classes=classes)
                          for prob, lab in zip(probas, labels))
    else:
        loss = lovasz_softmax_flat(*flatten_probas(probas, labels, ignore), classes=classes)
    return loss

def mean(l, ignore_nan=False, empty=0):
    """
    nanmean compatible with torch.tensor
    """
    l = iter(l)
    if ignore_nan:
        l = ifilterfalse(np.isnan, l)
    try:
        n = 1
        acc = next(l)
    except StopIteration:
        if empty == 'raise':
            raise
        return empty
    for n, v in enumerate(l, 2):
        acc += v
    if n == 1:
        return acc
    return acc / n

In [13]:
def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_seen = 0
    num_classes = 2
    total_correct_class = [0 for _ in range(num_classes)]
    total_seen_class = [0 for _ in range(num_classes)]
    total_iou_deno_class = [0 for _ in range(num_classes)]

    with torch.no_grad():
        for pts, lbls in dataloader:
            pts, lbls = pts.to(device), lbls.to(device)
            pts = pts.permute(0, 2, 1).float()  # [B, D, N]

            pred, _ = model(pts)                # [B, N, C]
            B, N, C = pred.shape

            pred_flat = pred.reshape(B * N, C)
            lbls_flat = lbls.reshape(B * N)

            loss_ce = criterion(pred_flat, lbls_flat)
            # loss_iou = iou_loss(pred_flat, lbls_flat) 
            # total_loss = loss_ce + 0.6 * loss_iou
            probas = F.softmax(pred_flat, dim=1)  # [B*N, C]
            loss_lovasz = lovasz_softmax_flat(probas, lbls_flat, classes='present') 
            total_loss = (0.6 * loss_ce + 0.4 * loss_lovasz).item()

            pred_labels = pred.argmax(dim=2)    # [B, N]
            correct = (pred_labels == lbls).sum().item()
            total_correct += correct
            total_seen += B * N

            # IoU подсчёт
            pred_np = pred_labels.cpu().numpy().reshape(-1)
            lbls_np = lbls.cpu().numpy().reshape(-1)

            for l in range(num_classes):
                total_seen_class[l] += np.sum(lbls_np == l)
                total_correct_class[l] += np.sum((pred_np == l) & (lbls_np == l))
                total_iou_deno_class[l] += np.sum((pred_np == l) | (lbls_np == l))

    acc = total_correct / total_seen
    iou_per_class = np.array(total_correct_class) / (np.array(total_iou_deno_class) + 1e-6)
    miou = np.mean(iou_per_class)
    avg_loss = total_loss / len(dataloader)
    model.train()
    return avg_loss, acc, miou, iou_per_class


In [14]:
# val_pairs[1][0]

In [15]:
train_dataset = KittiDynamicDataset(train_pairs, num_points=16384, voxel_size=0.1)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

val_dataset = KittiDynamicDataset(val_pairs, num_points=16384, voxel_size=0.1, is_train=False) 
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
# criterion = get_loss().to(device)
# criterion = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]).to(device))
criterion = FocalLoss(alpha=0.75, gamma=2.0)  # alpha >0.5 для minority

# scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)  # По mIoU

In [ ]:
import tqdm
model.train()
best_miou = -float('inf')  
best_model_path = "/kaggle/working/best_model.pth"
for epoch in range(3):
    count = 0
    total_loss = 0
    losses_cnt = 0
    train_dataset.dynamic_ratio = max(0.3, 0.4 - epoch * 0.1) #0.7
    for pts, lbls in tqdm.tqdm(train_dataloader):
        pts, lbls = torch.Tensor(pts.to(device)), torch.Tensor(lbls.to(device))
        pts = pts.permute(0, 2, 1).float() 

        optimizer.zero_grad()

        pred, trans_feat = model(pts)       # pred: [B, N, C]
        B, N, C = pred.shape
        pred_flat = pred.reshape(B * N, C)     # [B*N, C]
        lbls_flat = lbls.reshape(B * N)        # [B*N]

        # 1. # Another loss vers (bc + active weights)
        # class_counts = torch.bincount(lbls_flat, minlength=2).float()
        # class_probs = class_counts / class_counts.sum()
        # weights = 1.0 / (class_probs + 1e-6)
        # weights = weights / weights.sum() * 2  # Нормализуйте на num_classes
        # loss = F.cross_entropy(pred_flat, lbls_flat, weight=weights)

        # 2. Focal + IOU
        # loss_ce = criterion(pred_flat, lbls_flat)
        # loss_iou = iou_loss(pred_flat, lbls_flat) 
        # loss = loss_ce + 0.7 * loss_iou  # λ=0.5 - 1.0

        # 3. Lovasz-Softmax loss
        loss_ce = criterion(pred_flat, lbls_flat)
        probas = F.softmax(pred_flat, dim=1)  # [B*N, C]
        loss_lovasz = lovasz_softmax_flat(probas, lbls_flat, classes='present')
        loss = 0.6 * loss_ce + 0.4 * loss_lovasz  


        # loss = criterion(pred_flat, lbls_flat)

        total_loss += loss.item()
        losses_cnt+=1
        count+=1
        if (count%10 == 0):
            print(f"Count: {count} | curr mean loss {total_loss/count}")
        loss.backward()
        optimizer.step()

    # print(f"Epoch {epoch} | Loss: {total_loss/len(train_dataloader)}")
    val_loss, val_acc, val_miou, iou_per_class = validate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1} | Dynamic Ratio: {train_dataset.dynamic_ratio} | Loss: {total_loss/len(train_dataloader)} | val_loss={val_loss:.4f} | acc={val_acc:.4f} | mIoU={val_miou:.4f}")


    if val_miou > best_miou:
        best_miou = val_miou
        torch.save(model.state_dict(), best_model_path)
        print(f"New best mIoU: {best_miou:.4f}, model saved to {best_model_path}")
    # scheduler.step(val_miou)


 43%|████▎     | 10/23 [17:07<29:28, 136.04s/it]

Count: 10 | curr mean loss 0.17167039960622787


 87%|████████▋ | 20/23 [33:34<04:05, 81.67s/it] 

Count: 20 | curr mean loss 0.16753641106188297


100%|██████████| 23/23 [43:01<00:00, 112.26s/it]


...

Count: 10 | curr mean loss 0.16431291848421098


 87%|████████▋ | 20/23 [34:02<03:04, 61.52s/it] 

Count: 20 | curr mean loss 0.1622612565755844


100%|██████████| 23/23 [44:54<00:00, 117.16s/it]


Epoch 3 | Dynamic Ratio: 0.3 | Loss: 0.16197507083415985 | val_loss=0.0635 | acc=0.9531 | mIoU=0.5094


In [17]:
torch.save(model.state_dict(), "/kaggle/working/final_model.pth")
print("Final model saved")

Final model saved


In [16]:
!ls

data_utils  provider.py		    test_semseg.py	     visualizer
LICENSE     README.md		    train_classification.py
log	    test_classification.py  train_partseg.py
models	    test_partseg.py	    train_semseg.py


In [16]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_sample(features, labels, sample_idx=0, color_by='rgb', figsize=(10, 8)):
    """
    Simple visualization of a point cloud sample.
    
    Args:
        features (np.array): (N, 6) - XYZ + RGB
        labels (np.array): (N,) - semantic labels
        sample_idx (int): For title
        color_by (str): 'rgb' or 'labels'
        figsize (tuple): Plot size
    """
    points = features[:, :3]  
    rgb = features[:, 3:]     
    

    label_colors = {
        0: [0.5, 0.5, 0.5],  
        1: [1.0, 0.0, 0.0],  
        
    }
    
    fig = plt.figure(figsize=figsize)
    
    # 3D plot
    ax = fig.add_subplot(111, projection='3d')
    if color_by == 'rgb':
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=rgb, s=2, alpha=0.6)
        ax.set_title(f'Sample {sample_idx}: 3D RGB')
    else:
        colors_plot = np.array([label_colors.get(int(l), [0.5, 0.5, 0.5]) for l in labels])
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=colors_plot, s=2, alpha=0.6)
        ax.set_title(f'Sample {sample_idx}: 3D by Labels')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()
    
    
    fig2d = plt.figure(figsize=(10, 8))
    ax2d = fig2d.add_subplot(111)
    if color_by == 'rgb':
        ax2d.scatter(points[:, 0], points[:, 1], c=rgb, s=1, alpha=0.6)
        ax2d.set_title(f'Sample {sample_idx}: Top-Down RGB')
    else:
        ax2d.scatter(points[:, 0], points[:, 1], c=colors_plot, s=1, alpha=0.6)
        ax2d.set_title(f'Sample {sample_idx}: Top-Down by Labels')
    
    ax2d.set_xlabel('X')
    ax2d.set_ylabel('Y')
    ax2d.set_aspect('equal')
    plt.show()
    

    unique, counts = np.unique(labels, return_counts=True)
    plt.figure(figsize=(6, 4))
    plt.bar(unique, counts, alpha=0.7)
    plt.title(f'Sample {sample_idx}: Label Counts')
    plt.xlabel('Label ID')
    plt.ylabel('Count')
    plt.show()

In [ ]:
for idx in range(1, 6):
    features, labels = val_dataset[idx]    
    # print(features[: :6].shape, labels.shape)
    # visualize_sample(features[:, :6], labels, color_by='rgb')
    visualize_sample(features[:, :6], labels, color_by='labels')

    # for i in range(len(train_dataset)):
    #     features, labels = train_dataset[i]
    #     visualize_sample(features[:, :6], labels, color_by='labels')


    model.eval()
    features, labels = val_dataset[idx]
    features = features.unsqueeze(0).permute(0, 2, 1).float().to(device)  # [1, 9, 16384]
    with torch.no_grad():
        pred, _ = model(features)
        pred_labels = pred.argmax(dim=2).cpu().numpy().reshape(-1)  # [16384]
        # print(pred_labels[:10])
    # print(features[0].permute(1, 0).shape, pred_labels.shape) # [16384, 9], [16384]
    visualize_sample(features[0].permute(1, 0).cpu().numpy(), pred_labels, color_by='labels')

In [ ]:
import os
import json
import subprocess
from pathlib import Path

KAGGLE_JSON_PATH = '/kaggle/input/kaggle2/kaggle.json'  

!mkdir -p /root/.kaggle
!cp {KAGGLE_JSON_PATH} /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

torch.save(model.state_dict(), '/kaggle/working/final_model.pth')

dataset_metadata = {
    "title": "My-Trained-Model-Weights-17",
    "id": "us/lidarSegmModel-17",  
    "licenses": [{"name": "CC0-1.0"}],
    "description": "Trained model weights from my project"
}

with open('/kaggle/working/dataset-metadata.json', 'w') as f:
    json.dump(dataset_metadata, f)

result = subprocess.run([
    'kaggle', 'datasets', 'create',
    '-p', '/kaggle/working',
    '--dir-mode', 'zip'
], capture_output=True, text=True)

print("Output:", result.stdout)
print("Errors:", result.stderr)

Output: Starting upload for file best_model.pth
Upload successful: best_model.pth (4MB)
Starting upload for file final_model.pth
Upload successful: final_model.pth (4MB)
Starting upload for file Pointnet_Pointnet2_pytorch.zip
Upload successful: Pointnet_Pointnet2_pytorch.zip (137MB)
Starting upload for file .virtual_documents.zip
Upload successful: .virtual_documents.zip (22B)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/vadimcvbnqq/lidarSegmModel-17

Errors: 
  0%|          | 0.00/3.77M [00:00<?, ?B/s]
100%|██████████| 3.77M/3.77M [00:00<00:00, 14.4MB/s]

  0%|          | 0.00/3.77M [00:00<?, ?B/s]
100%|██████████| 3.77M/3.77M [00:00<00:00, 16.7MB/s]

  0%|          | 0.00/137M [00:00<?, ?B/s]
 17%|█▋        | 22.7M/137M [00:00<00:00, 226MB/s]
 32%|███▏      | 44.3M/137M [00:00<00:00, 191MB/s]
 46%|████▌     | 62.8M/137M [00:00<00:00, 170MB/s]
 58%|█████▊    | 79.3M/137M [00:00<00:00, 159MB/s]
 69%|██████▉   | 94.6M/137M [00:00<00:00,